In [1]:
staticDataFrame = spark.read.format("csv").option("header", "true") \
        .option("inferSchema", "true")\
        .load("/FileStore/tables/2011*.csv")




In [2]:
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [3]:
from pyspark.sql.functions import window, column, desc, col

staticDataFrame\
.selectExpr(    "CustomerId",    "(UnitPrice * Quantity) as total_cost",    "InvoiceDate")\
.groupBy(    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")\
.show(5)

+----------+--------------------+-----------------+
CustomerId| window| sum(total_cost)|
+----------+--------------------+-----------------+
 16057.0|[2011-12-05 00:00...| -37.6|
 14126.0|[2011-11-29 00:00...|643.6300000000001|
 13500.0|[2011-11-16 00:00...|497.9700000000001|
 17160.0|[2011-11-08 00:00...|516.8499999999999|
 15608.0|[2011-11-11 00:00...| 122.4|
+----------+--------------------+-----------------+
only showing top 5 rows

In [4]:
# in Python
streamingDataFrame = spark.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger", 1)\
.format("csv")\
.option("header", "true")\
.load("/FileStore/tables/2011*.csv")

In [5]:
streamingDataFrame.isStreaming

Out[8]: True

In [6]:
# in Python
purchaseByCustomerPerHour = streamingDataFrame\
.selectExpr(    "CustomerId",    "(UnitPrice * Quantity) as total_cost",    "InvoiceDate")\
.groupBy(    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")

In [7]:
purchaseByCustomerPerHour.writeStream\
.format("memory")\
.queryName("customer_purchases")\
.outputMode("complete")\
.start()

Out[10]: <pyspark.sql.streaming.StreamingQuery at 0x7fd69ac52978>

In [8]:
# in Python
spark.sql("""  SELECT *  FROM customer_purchases  ORDER BY `sum(total_cost)` DESC  """)\
.show(5)

+----------+--------------------+------------------+
CustomerId| window| sum(total_cost)|
+----------+--------------------+------------------+
 null|[2011-12-08 00:00...|31975.590000000007|
 null|[2011-11-29 00:00...|23744.250000000055|
 null|[2011-11-30 00:00...| 18630.12999999988|
 null|[2011-12-09 00:00...|16571.320000000036|
 null|[2011-12-02 00:00...|12950.379999999943|
+----------+--------------------+------------------+
only showing top 5 rows

In [9]:
purchaseByCustomerPerHour.writeStream\
.format("console")\
.queryName("customer_purchases_2")\
.outputMode("complete")\
.start()

Out[12]: <pyspark.sql.streaming.StreamingQuery at 0x7fd6abab2f28>

In [10]:
# in Python
from pyspark.sql.functions import date_format, col
preppedDataFrame = staticDataFrame\
.na.fill(0)\
.withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
.coalesce(5)

In [11]:
preppedDataFrame.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
 580538| 23084| RABBIT NIGHT LIGHT| 48|2011-12-05 08:38:00| 1.79| 14075.0|United Kingdom| Monday|
 580538| 23077| DOUGHNUT LIP GLOSS | 20|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom| Monday|
 580538| 22906|12 MESSAGE CARDS ...| 24|2011-12-05 08:38:00| 1.65| 14075.0|United Kingdom| Monday|
 580538| 21914|BLUE HARMONICA IN...| 24|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom| Monday|
 580538| 22467| GUMBALL COAT RACK| 6|2011-12-05 08:38:00| 2.55| 14075.0|United Kingdom| Monday|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
only showing top 5 rows